In [1]:
%matplotlib inline
import csv
import numpy as np
import pandas as pd
from scipy.signal import *
import matplotlib.pyplot as plt
from scipy.ndimage import interpolation
from scipy.stats import entropy

# k Nearest Neighbors Classifier
from sklearn.neighbors import KNeighborsClassifier

import time

largeur=15
hauteur=6

In [4]:
step_type_array = ["brush_back","brush_forward","dig","heel","step"]
size_array = [29,29,29,29,29]
ext = ".txt"
dancer = "daniel"

columns=['step type','g0x', 'g0y', 'g0z', 'a0x', 'a0y', 'a0z',
                     'g1x', 'g1y', 'g1z', 'a1x', 'a1y', 'a1z']

array_feat = pd.DataFrame(columns= columns)


for i in range(len(size_array)):
    step_type = step_type_array[i]
    size = size_array[i]
    
    path_prep =  "etapdance2" + "/" + dancer + "/" + step_type + "/" + step_type + "_cut_int" + "/" + step_type + "_cut_int_"

    for n in range(size+1):
        
        step_type_nb = str(n)
        ext = ".txt"
        data_input = path_prep + step_type_nb + ext

        data_raw = pd.read_csv(data_input,names=['Gyro0X (deg)','Gyro0Y (deg)','Gyro0Z (deg)','Accel0X (g)','Accel0Y (g)','Accel0Z (g)','Gyro1X (deg)','Gyro1Y (deg)','Gyro1Z (deg)','Accel1X (g)','Accel1Y (g)','Accel1Z (g)'], dtype='a') 
        
        # MPU0 6050 (talon)
        g0x = np.array(data_raw['Gyro0X (deg)'])[1:].astype(float)
        g0y = np.array(data_raw['Gyro0Y (deg)'])[1:].astype(float)
        g0z = np.array(data_raw['Gyro0Z (deg)'])[1:].astype(float)

        a0x = np.array(data_raw['Accel0X (g)'])[1:].astype(float)
        a0y = np.array(data_raw['Accel0Y (g)'])[1:].astype(float)
        a0z = np.array(data_raw['Accel0Z (g)'])[1:].astype(float)

        # MPU1 6050 (pointe)
        g1x = np.array(data_raw['Gyro1X (deg)'])[1:].astype(float)
        g1y = np.array(data_raw['Gyro1Y (deg)'])[1:].astype(float)
        g1z = np.array(data_raw['Gyro1Z (deg)'])[1:].astype(float)

        a1x = np.array(data_raw['Accel1X (g)'])[1:].astype(float)
        a1y = np.array(data_raw['Accel1Y (g)'])[1:].astype(float)
        a1z = np.array(data_raw['Accel1Z (g)'])[1:].astype(float)
        #print(step_type + str(n))
        g0_max = max(abs(g0y))
        a0_max = max(abs(a0z))
        g1_max = max(abs(g1y))
        a1_max = max(abs(a1z))

        g0x_uint8 = (g0x/max(abs(g0x))*127+127).astype('uint8')
        g0y_uint8 = (g0y/max(abs(g0y))*127+127).astype('uint8')
        g0z_uint8 = (g0z/max(abs(g0z))*127+127).astype('uint8')
        
        a0x_uint8 = (a0x/max(abs(a0x))*127+127).astype('uint8')
        a0y_uint8 = (a0y/max(abs(a0x))*127+127).astype('uint8')
        a0z_uint8 = (a0z/max(abs(a0x))*127+127).astype('uint8')
        
        g1x_uint8 = (g1x/max(abs(g1x))*127+127).astype('uint8')
        g1y_uint8 = (g1y/max(abs(g1y))*127+127).astype('uint8')
        g1z_uint8 = (g1z/max(abs(g1z))*127+127).astype('uint8')
        
        a1x_uint8 = (a1x/max(abs(a1x))*127+127).astype('uint8')
        a1y_uint8 = (a1y/max(abs(a1y))*127+127).astype('uint8')
        a1z_uint8 = (a1z/max(abs(a1z))*127+127).astype('uint8')
        
        array_feat_temp = [[step_type, g0x_uint8, g0y_uint8, g0z_uint8, a0x_uint8, a0y_uint8, a0z_uint8, 
                                       g1x_uint8, g1y_uint8, g1z_uint8, a1x_uint8, a1y_uint8, a1z_uint8]]
        array_feat_temp = pd.DataFrame(array_feat_temp,columns=columns)
        array_feat = array_feat.append(array_feat_temp,ignore_index =True, sort =False)
        
        
array_feat

labels = array_feat['step type'].apply(pd.Series)

g0x_df = array_feat['g0x'].apply(pd.Series).astype('uint8')
g0y_df = array_feat['g0y'].apply(pd.Series).astype('uint8')
g0z_df = array_feat['g0z'].apply(pd.Series).astype('uint8')

a0x_df = array_feat['a0x'].apply(pd.Series).astype('uint8')
a0y_df = array_feat['a0y'].apply(pd.Series).astype('uint8')
a0z_df = array_feat['a0z'].apply(pd.Series).astype('uint8')

g1x_df = array_feat['g1x'].apply(pd.Series).astype('uint8')
g1y_df = array_feat['g1y'].apply(pd.Series).astype('uint8')
g1z_df = array_feat['g1z'].apply(pd.Series).astype('uint8')

a1x_df = array_feat['a1x'].apply(pd.Series).astype('uint8')
a1y_df = array_feat['a1y'].apply(pd.Series).astype('uint8')
a1z_df = array_feat['a1z'].apply(pd.Series).astype('uint8')


dataset = pd.concat([labels[:], g0x_df[:], g0y_df[:], g0z_df[:], a0x_df[:], a0y_df[:], a0z_df[:], 
                                g1x_df[:], g1y_df[:], g1z_df[:], a1x_df[:], a1y_df[:], a1z_df[:]], axis=1)

dataset

dataset.to_csv("etapdance2/dataset_cut_daniel_data_only.txt",index=False)

In [ ]:
plt.figure(figsize=(largeur,hauteur))
        plt.subplot(1, 2, 1)
        plt.plot(g0x_uint8,label='Gyro0X uint8')
        plt.plot(g0y_uint8,label='Gyro0Y uint8')
        plt.plot(g0z_uint8,label='Gyro0Z uint8')
        plt.plot(g1x_uint8,label='Gyro1X uint8')
        plt.plot(g1y_uint8,label='Gyro1Y uint8')
        plt.plot(g1z_uint8,label='Gyro1Z uint8')
        plt.ylabel('amplitude (-)')
        plt.xlabel('échantillons (-)')
        plt.autoscale(axis='x',tight=True);
        plt.legend()
        plt.title(data_input)
        plt.subplot(1, 2, 2)
        plt.plot(a0x_uint8,label='Accel0X uint8')
        plt.plot(a0y_uint8,label='Accel0Y uint8')
        plt.plot(a0z_uint8,label='Accel0Z uint8')
        plt.plot(a1x_uint8,label='Accel1X uint8')
        plt.plot(a1y_uint8,label='Accel1Y uint8')
        plt.plot(a1z_uint8,label='Accel1Z uint8')
        plt.ylabel('amplitude (-)')
        plt.xlabel('échantillons (-)')
        plt.autoscale(axis='x',tight=True);
        plt.legend()
        plt.show()